In [43]:
from os import supports_follow_symlinks
import nltk
import csv
import pathlib
import json
import numpy as np


nltk.download('punkt')

file_list = ['train', 'test','dev']
subject = ['Movies_1', 'Movies_2', 'Movies_3']

[nltk_data] Downloading package punkt to /home/dgmneto/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [64]:
list_entities = {"":[]}
def get_entities(frames):
    for frame in frames:
        for action in frame['actions']:
            if action['slot'] == 'intent':
                continue
            values = action['values']
            for value in values:
                entity = action['slot']
                if entity not in list_entities:
                    list_entities[entity] = []
                list_entities[entity].append(value)
                
def get_entity_list(json_file):
    phrase_to_entities = []
    for item in json_file:
        if len(item['services']) > 1 or not np.any(np.in1d(subject, item['services'])):
            continue
        for turn in item['turns']:
            if turn['speaker'] == 'SYSTEM':
                continue
            splitted_text = nltk.word_tokenize(turn['utterance'])
            get_entities(turn['frames'])

In [68]:
for file_name in file_list:
    for path in pathlib.Path(f'../dstc8-schema-guided-dialogue/{file_name}').iterdir():
        if path.is_file() and not path.match('*schema.json'):
            file = open(path, 'r')
            json_file = json.loads(file.read())
            get_entity_list(json_file)
    file.close()
for entity in list_entities:
    set_values = set(list_entities[entity])
    for value in set_values:
        f = open(f'./extraction_rasa/{entity}_rasa.txt', 'a')
        f.write(str(value + '\n'))
        f.close()
        
